### Resume Parser by Regular Expression

In [23]:
import regex as re

In [ ]:
!pip install pdfminer.six

In [ ]:
!pip install docx2txt

In [ ]:
!pip install nlp

In [ ]:
!pip install spacy

**Extracting Text from PDF**

In [28]:
import pdfminer
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import docx2txt
import os

In [ ]:
rootdir = '/content/drive/MyDrive/DataForMasterProject/ActualDataFolder/RegExData'
files = os.listdir(rootdir)
file_names = []
for file in os.listdir(rootdir):
    file_names.append(os.path.join(rootdir, file))
file_names

In [10]:
def open_pdf_file(file_name):
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    pagenums = set()
    infile = open(file_name, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close()

    result = []

    for line in text.split('\n'):
        line2 = line.strip()
        if line2 != '':
            result.append(line2)
    return (result)

In [30]:
def open_docx_file(file_name):
    temp = docx2txt.process(file_names[0])
    text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
    text = [t for t in text if len(t) > 1]
    return (text)

In [31]:
def remove_punctuations(line):
    return re.sub(r'(\.|\,)', '', line)

def preprocess_document(document):
    for index, line in enumerate(document):
        line = line.lower()
        line = remove_punctuations(line)

        line = line.split(' ')
        while '' in line:
            line.remove('')

        while ' '  in line:
            line.remove(' ')


        document[index] = ' '.join(line)
    return (document)

In [32]:
#Name
import spacy
from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)

    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]

    matcher.add('NAME', [pattern], on_match = None)

    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text


#name = extract_name(text)
#print(name)
#extracted_text["Name"] = name

In [33]:
def get_email(document):
    #Further optimization to be done.
    emails = []
    pattern = re.compile(r'\w+@[a-zA-Z_]+?\.[a-zA-Z]{2,3}')
    for line in document:
        matches = pattern.findall(line)
        for mat in matches:
            if len(mat) > 0:
                emails.append(mat)
    #print (emails)
    return (emails)

In [34]:
def get_phone_no(document):
    #This function has to be further modified better and accurate results.
    #Possible phone number formats - Including +91 or just with the numbers.

    mob_num_regex = r'''(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)
                        [-\.\s]*\d{3}[-\.\s]??\d{4}|\d{5}[-\.\s]??\d{4})'''
    pattern = re.compile(mob_num_regex)
    matches = []
    for line in document:
        match = pattern.findall(line)
        for mat in match:
            if len(mat) > 9:
                matches.append(mat)

    return (matches)

In [35]:
EDU_PATH='/content/drive/MyDrive/DataForMasterProject/ActualDataFolder/education.txt'
def get_education(document):
    education_terms = []
    with open(EDU_PATH, 'r') as file:
        education_terms = file.readlines()

    education_terms = [term.strip('\n') for term in education_terms]
    education = []
    for line in document:
        for word in line.split(' '):
            if len(word) > 2 and word in education_terms:
                if line not in education:
                    education.append(line)
    #print (education)
    return (education)

In [36]:
SKILL_PATH='/content/drive/MyDrive/DataForMasterProject/ActualDataFolder/skills.txt'
def get_skills(document):
    skill_terms = []
    with open(SKILL_PATH, 'r') as file:
        skill_terms = file.readlines()

    skill_terms = [term.strip('\n') for term in skill_terms]
    skills = []

    for line in document:
        words = line.split(' ')

        for word in words:
            if word in skill_terms:
                if word not in skills:
                    skills.append(word)

        word_pairs = []
        for i in zip(words[:-1], words[1:]):
            word_pairs.append(i[0] + ' ' + i[1])   #This is to find skills like 'data science' i.e skills containint two words.    return (skills)

        for pair in word_pairs:
            if pair in skill_terms:
                if pair not in skills:
                    skills.append(pair)

    return (skills)

In [37]:
def get_experience(document):
    pattern1 = re.compile(r'(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4}).*(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4})')
    pattern2 = re.compile(r'(\d{2}(.|..)\d{4}).{1,4}(\d{2}(.|..)\d{4})')
    pattern3 = re.compile(r'(\d{2}(.|..)\d{4}).{1,4}(present)')
    pattern4 = re.compile(r'(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4}).*(present)')
    patterns = [pattern1, pattern2, pattern3, pattern4]
    experience = []
    for index, line in enumerate(document):
        for pattern in patterns:
            exp = pattern.findall(line)
            if len(exp) > 0:
                experience.append(document[index:index+4])

    return (experience)

In [38]:
file_name='/content/drive/MyDrive/DataForMasterProject/ActualDataFolder/RegExData/resume1.pdf'
document = open_pdf_file(file_name)

print(document)

['SURYAKIRAN KASTURI', 'Contact: +91-9448821250/+916362463338 Email: kirankss@gmail.com; Date of Birth: 28th Dec 1975', 'Product Development Leader ~ Full stack ~ Process Automation', 'Offering prolific experience in the domain of software designing, testing, implementation and support; seeking senior level', 'assignments across the industry, while bringing the following deliverables:', 'Strategic Planning ~ Performance Improvement ~ Full Product Lifecycle ~ Cloud ~ Analytics', '~ Diverse Technologies ~ Mobile Apps ~ Mentoring and coaching teams ~ CI/CD ~ TDD', '\uf0fc  Presenter at eXtreme Programming Conference, India 2015 on TDD in embedded environment', '\uf0fc  Member of the Core Agile team at IDC and organizer of the first Agile Day at Intuit', '\uf0fc  Member of the innovation council and have organized innovation events at Allstate', 'Career Precis', '\uf0fc  Product Development Leader with experience in leading teams while being an IC during design and development', '\uf0fc  E

In [47]:
name_list =[]
email_ids = []
phone_nos = []
education_1 = []
education_2 = []
skills_1 = []
skills_2 = []
experience_1 = []
experience_2 = []

for file_name in file_names:
    if file_name.endswith('.pdf'):
        document = open_pdf_file(file_name)
    elif file_name.endswith('.docx'):
        document = open_docx_file(file_name)

    my_string = ' '.join(document)
    name = extract_name(my_string)
    print(name)
    if len(name) > 0:
        name_list.append(name)
    else:
        name_list.append('')


    email = get_email(document)
    phone_no = get_phone_no(document)
    document = preprocess_document(document)
    #print ('\n\n')
    #print (file_name)
    #print ('Email is {} phone number is {}'.format(email, phone_no))
    if len(email) > 0:
        email_ids.append(email[0])
    else:
        email_ids.append('')

    if len(phone_no) > 0:
        phone_nos.append(phone_no[0])
    else:
        phone_nos.append('')

    education = get_education(document)
    #print ('Education ', get_education(document))
    if len(education) > 1:
        education_1.append(education[0])
        education_2.append(education[1])
    elif len(education) == 1:
        education_1.append(education[0])
        education_2.append('')
    elif len(education) == 0:
        education_1.append('')
        education_2.append('')

    skills = get_skills(document)
    #print ('Skills ', skills)

    if len(skills) > 1:
        skills_1.append(skills[0])
        skills_2.append(skills[1])
    elif len(skills) == 1:
        skills_1.append(skills[0])
        skills_2.append('')
    elif len(skills) == 0:
        skills_1.append('')
        skills_2.append('')

    experience = get_experience(document)
    #print ('Experience ', get_experience(document))
    if len(experience) > 1:
        experience_1.append(experience[0])
        experience_2.append(experience[1])
    elif len(experience) == 1:
        experience_1.append(experience[0])
        experience_2.append('')
    elif len(experience) == 0:
        experience_1.append('')
        experience_2.append('')

SURYAKIRAN KASTURI
MONICA MEYER
McAllister St
Sébastien Genty
Brendan Herger
John Smith
john smith


In [48]:
import pandas as pd
df = pd.DataFrame({'Name':name_list,'email id':email_ids, 'phone nos':phone_nos, 'education 1':education_1, 'education 2':education_2, 'skills 1':skills_1,
              'skills 2':skills_2, 'experience 1':experience_1, 'experiece 2':experience_2})

In [49]:
!pip install openpyxl

In [50]:
PATH_SAVE='/content/drive/MyDrive/DataForMasterProject/ActualDataFolder/Resume_extracted_data.xlsx'
#df.to_excel(PATH_SAVE)

In [51]:
try:
    df.to_excel(PATH_SAVE)
    print("File saved successfully.")
except Exception as e:
    print("Error occurred while saving the file:", str(e))

File saved successfully.
